In [1]:
!pip install bert-extractive-summarizer

  Using cached spacy-3.5.2-cp310-cp310-win_amd64.whl (12.2 MB)
  Using cached srsly-2.4.6-cp310-cp310-win_amd64.whl (480 kB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached pydantic-1.10.7-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached murmurhash-1.0.9-cp310-cp310-win_amd64.whl (18 kB)
     ---------------------------------------- 1.5/1.5 MB 7.8 MB/s eta 0:00:00
  Using cached pathy-0.10.1-py3-none-any.whl (48 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached preshed-3.0.8-cp310-cp310-win_amd64.whl (94 kB)
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached confection-0.0.4-py3-none-any.whl (32 kB)
  Using cached blis-0.7.9-cp310-cp310-win_amd64.whl (7.0 MB)


In [2]:
!pip install transformers

     -------------------------------------- 360.6/360.6 kB 3.7 MB/s eta 0:00:00
     ------------------------------------- 977.5/977.5 kB 10.3 MB/s eta 0:00:00
  Using cached sacremoses-0.0.53-py3-none-any.whl
     -------------------------------------- 135.6/135.6 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 79.8/79.8 kB ? eta 0:00:00
     ---------------------------------------- 10.7/10.7 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bert-score 0.3.13 requires transformers>=3.0.0, but you have transformers 2.2.0 which is incompatible.


In [3]:
!pip install spacy==2.0.12


     ---------------------------------------- 22.0/22.0 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 112.7/112.7 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.2/1.2 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 150.7/150.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 601.6/601.6 kB 5.4 MB/s eta 0:0

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [632 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\spacy
  copying spacy\about.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\compat.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\errors.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\glossary.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\language.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\lemmatizer.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\scorer.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\util.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\_ml.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\__init__.py -> build\lib.win-amd64-cpython-310\spacy
  copying spacy\__mai

In [1]:
#from summarizer import Summarizer,TransformerSummarizer

In [ ]:
import pandas as pd
from summarizer import Summarizer, TransformerSummarizer

# Define the summarization models
models = {"BERT": Summarizer(), 
          "GPT-2": TransformerSummarizer(transformer_type="GPT2", transformer_model_key="gpt2-medium"), 
          "XLNet": TransformerSummarizer(transformer_type="XLNet", transformer_model_key="xlnet-base-cased")}

# Load the Excel file
df = pd.read_excel("first150.xlsx")

# Loop through each row in the dataframe
for index, row in df.iterrows():
    # Extract the text from the current row
    text = row["article"]
    
    # Loop through each summarization model
    for model_name, model in models.items():
        # Summarize the text using the current model
        summary = ''.join(model(text, min_length=60))
        
        # Append the summary to the new column
        df.at[index, f"{model_name} summary"] = summary

# Save the updated dataframe to a new Excel file
df.to_excel("output_first150.xlsx", index=False)

In [7]:
!pip install rouge_score

In [1]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the Excel file
df = pd.read_excel('output_first50.xlsx')

# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Apply the scorer to the existing summary column for each model
for name in ['chatgpt summary', 'BERT summary', 'GPT-2 summary','XLNet summary']:
    summary_column = f'{name}'
    scores_column_prefix = f'{name} - '
    rouge_scores = df.apply(lambda row: scorer.score(row['manual summary'], row[summary_column]), axis=1)
    df[f'{scores_column_prefix}rouge1'] = rouge_scores.apply(lambda score: score['rouge1'].fmeasure)
    df[f'{scores_column_prefix}rouge2'] = rouge_scores.apply(lambda score: score['rouge2'].fmeasure)
    df[f'{scores_column_prefix}rougeL'] = rouge_scores.apply(lambda score: score['rougeL'].fmeasure)

# Save the results to the same Excel file
#df.to_excel('scores.xlsx', index=False)


In [ ]:
!pip install bert_score

In [ ]:
import pandas as pd
import bert_score

# Load data from excel
df = pd.read_excel('output_first50.xlsx')

# Reference summary
ref_summary = 'manual summary'

# Candidate summaries
candidate_summaries = ['chatgpt summary', 'BERT summary', 'GPT-2 summary','XLNet summary']

# Calculate the BERTScore for each candidate summary
for cand_summary in candidate_summaries:
    # Initialize a list to store the scores
    bert_scores = []
    # Iterate through each row of the dataframe
    for index, row in df.iterrows():
        # Get the reference summary and candidate summary
        ref = row[ref_summary]
        candidate = row[cand_summary]
        # Calculate the BERTScore using bert_score
        P, R, F1 = bert_score.score([candidate], [ref], lang='en', verbose=False)
        # Add the F1 score to the list
        bert_scores.append(F1.item())
    # Add the scores to the dataframe
    df[cand_summary + ' BERTScore'] = bert_scores

# Save the updated dataframe to excel
df.to_excel('updated_data.xlsx', index=False)

In [ ]:
import pandas as pd
import bert_score

# Load data from excel
df = pd.read_excel('output_data1.xlsx')

# Reference summary
ref_summary = 'manual summary'

# Candidate summaries
candidate_summaries = ['chatgpt summary', 'BERT summary', 'GPT-2 summary','XLNet summary']

# Calculate the BERTScore for each candidate summary
for cand_summary in candidate_summaries:
    # Initialize a list to store the scores
    bert_scores = []
    # Get the reference summaries
    refs = df[ref_summary].tolist()
    # Get the candidate summaries
    candidates = df[cand_summary].tolist()
    # Calculate the BERTScore using bert_score
    P, R, F1 = bert_score.score(candidates, refs, lang='en', verbose=False)
    # Add the F1 scores to the list
    bert_scores = F1.tolist()
    # Add the scores to the dataframe
    df[cand_summary + ' BERTScore'] = bert_scores

# Save the updated dataframe to excel
df.to_excel('output_data2.xlsx', index=False)